# Wunderground Weather Condition to Weather Code Translator

Full document available from *"The Weather Company"*:

https://docs.google.com/document/d/1qpc4QN3YDpGDGGNYVINh7tfeulcZ4fxPSC5f4KzpR_U/edit

## Prerequisites

Import all required packages.

In [1]:
# Import required packages.

# Import pandas for data analysis.
import pandas as pd

# Import datetime for use with the "datetime" feature.
import datetime as dt

# Import numpy for numeric computing.
import numpy as np


Read the .csv file to be edited.

In [2]:
# Read the Wunderground .csv file.
df = pd.read_csv("historical_weather_data.csv", keep_default_na = True, delimiter = ",", skipinitialspace = True, encoding = "Windows-1252")

# Write to file using default UTF-8 encoding.
df.to_csv("historical_weather_data.csv", index = False)

## Brief Data Quality Report

Ensure that the file has read correctly and presents as expected. Perform basic sanity checking, and finally check for any missing columns.

In [3]:
# Check how many rows and columns the dataset has.
df.shape

(13110, 11)

In [4]:
# Print the first five rows.
df.head(5)

,timestamp,temperature_f,dew_point_f,humidity_%,wind,wind_speed_mph,wind_gust_mph,pressure_in,precip_in,weather_condition,datetime
0,1676508660,56,50,80,S,3,0,29.79,0.0,Cloudy,2023-02-16 00:51:00
1,1676512260,61,39,44,SW,16,24,29.91,0.0,Partly Cloudy,2023-02-16 01:51:00
2,1676515860,61,39,44,WSW,16,23,29.93,0.0,Mostly Cloudy,2023-02-16 02:51:00
3,1676519460,60,38,44,WSW,15,0,29.93,0.0,Mostly Cloudy,2023-02-16 03:51:00
4,1676523060,60,38,44,SW,13,0,29.94,0.0,Mostly Cloudy,2023-02-16 04:51:00


In [5]:
# Print the last five rows.
df.tail(5)

,timestamp,temperature_f,dew_point_f,humidity_%,wind,wind_speed_mph,wind_gust_mph,pressure_in,precip_in,weather_condition,datetime
13105,1716751800,72,65,78,ESE,8,0,29.89,0.0,Cloudy,2024-05-26 19:30:00
13106,1716753060,72,65,78,ESE,9,0,29.89,0.0,Cloudy,2024-05-26 19:51:00
13107,1716756660,70,65,84,SSE,7,0,29.90,0.0,Cloudy,2024-05-26 20:51:00
13108,1716760260,71,65,81,S,9,0,29.89,0.0,Cloudy,2024-05-26 21:51:00
13109,1716763860,68,64,87,SSE,13,0,29.90,0.0,Mist,2024-05-26 22:51:00


In [6]:
# Search for constant columns.
df.nunique()

timestamp            13104
temperature_f           78
dew_point_f             80
humidity_%              88
wind                    18
wind_speed_mph          34
wind_gust_mph           36
pressure_in            168
precip_in               14
weather_condition       45
datetime             13104
dtype: int64

In [7]:
# List data types as determined by pandas.
df.dtypes

timestamp              int64
temperature_f          int64
dew_point_f            int64
humidity_%             int64
wind                  object
wind_speed_mph         int64
wind_gust_mph          int64
pressure_in          float64
precip_in            float64
weather_condition     object
datetime              object
dtype: object

Three columns are shown to be of class "object", those being:
- wind
- weather_condition
- datetime

Should these be used in the model, they can be converted to int64 or float64 in the case of *"wind"* and *"weather_condition"*, and datetime in the case of *"datetime"*.

It is recommended that all numerical values are converted to float64 for consistency.

In [8]:
# Change all data fields to either continuous (float64) types, or to datetime form.
df["timestamp"] = df["timestamp"].astype("float64")
df["temperature_f"] = df["temperature_f"].astype("float64")
df["dew_point_f"] = df["dew_point_f"].astype("float64")
df["humidity_%"] = df["humidity_%"].astype("float64")
df["wind_speed_mph"] = df["wind_speed_mph"].astype("float64")
df["wind_gust_mph"] = df["wind_gust_mph"].astype("float64")
df["datetime"] = pd.to_datetime(df["datetime"])
df["datetime"] = df["datetime"].map(dt.datetime.toordinal)

In [9]:
# List the updated data types as determined by pandas.
df.dtypes

timestamp            float64
temperature_f        float64
dew_point_f          float64
humidity_%           float64
wind                  object
wind_speed_mph       float64
wind_gust_mph        float64
pressure_in          float64
precip_in            float64
weather_condition     object
datetime               int64
dtype: object

## Translating *"weather_condition"* to the Relevant Weather Code

In [10]:
# Investigate what unique values there are in the column, finding their length, and ordering them in a list.
print("List of unique values in column:", df["weather_condition"].unique())

List of unique values in column: ['Cloudy' 'Partly Cloudy' 'Mostly Cloudy' 'Fair' 'Light Rain'
 'Mostly Cloudy / Windy' 'Rain / Windy' 'Heavy Rain / Windy'
 'Light Rain / Windy' 'Cloudy / Windy' 'Wintry Mix / Windy'
 'Partly Cloudy / Windy' 'Fair / Windy' 'Rain' 'Light Rain with Thunder'
 'T-Storm' 'Wintry Mix' 'Light Drizzle' 'Fog' 'Haze' 'Light Snow'
 'Light Snow and Sleet' 'Snow and Sleet' 'Snow' 'Light Snow / Windy'
 'Light Drizzle / Windy' 'T-Storm / Windy' 'Heavy Rain' 'Drizzle and Fog'
 'Thunder' 'Thunder in the Vicinity' 'Haze / Windy' 'Smoke / Windy'
 'Smoke' 'Heavy T-Storm' 'Mist' 'Heavy T-Storm / Windy' 'Heavy Rain / Fog'
 'Mist / Windy' 'Light Freezing Rain' 'Snow / Fog' 'Heavy Snow / Fog'
 'Light Rain / Fog' 'Thunder and Hail / Fog' 'Fog / Windy']


In [11]:
weather_codes = {
    "Tornado" : 0,
    "Tropical Storm" : 1,
    "T-Storm" : 1, # Alias.
    "Hurricane" : 2,
    "Strong Storms" : 3,
    "Thunderstorms" : 4,
    "Thunder" : 4, # Alias.
    "Thunder in the Vicinity" : 4, # Alias.
    "Rain and Snow" : 5, # Alias (removed forward slash (/)).
    "Rain and Sleet" : 6, # Alias (removed forward slash (/).
    "Wintry Mix" : 7,
    "Freezing Drizzle" : 8,
    "Drizzle" : 9,
    "Freezing Rain" : 10,
    "Showers" : 11,
    "Rain" : 12,
    "Flurries" : 13,
    "Snow Showers" : 14,
    "Blowing Snow" : 15, # Alias (removed forward slash (/)).
    "Drifting Snow" : 15, # Alias (removed forward slash (/)).
    "Snow" : 16,
    "Hail" : 17,
    "Sleet" : 18,
    "Blowing Dust" : 19, # Alias (removed forward slash (/)).
    "Sandstorm": 19, # Alias (removed forward slash (/)).
    "Foggy" : 20,
    "Fog" : 20, # Alias.
    "Haze" : 21,
    "Smoke" : 22,
    "Breezy" : 23,
    "Windy" : 24,
    "Frigid" : 25, # Alias (removed forward slash (/)).
    "Ice Crystals" : 25, # Alias (removed forward slash (/)).
    "Cloudy" : 26,
    "Mostly Cloudy" : 27, # Duplicates with 28.
    "Partly Cloudy" : 29, # Duplicates with 30.
    "Clear" : 31,
    "Sunny" : 32,
    "Fair" : 33, # Duplicates with 34.
    "Mixed Rain and Hail" : 35,
    "Hot" : 36,
    "Isolated Thunderstorms" : 37,
    "Scattered Thunderstorms" : 38, # Duplicates with 47.
    "Scattered Showers" : 39, # Duplicates with 45.
    "Heavy Rain" : 40,
    "Scattered Snow Showers" : 41, # Duplicates with 46.
    "Heavy Snow" : 42,
    "Blizzard" : 43,
    "Not Available (NA)" : 44,
    "Snow and Sleet" : 50, # Not included in the original document.
    "Light Snow and Sleet" : 51, # Not included in the original document.
    "Light Drizzle" : 90, # Not included in the original document.
    "Heavy Drizzle" : 91, # Not included in the original document.
    "Light Freezing Drizzle" : 92, # Not included in the original document.
    "Heavy Freezing Drizzle" : 93, # Not included in the original document.
    "Drizzle and Fog" : 94, # Not included in the original document.
    "Heavy T-Storm" : 100, # Not included in the original document.
    "Light Rain" : 120, # Not included in the original document.
    "Heavy Rain" : 121, # Not included in the original document.
    "Light Freezing Rain" : 122, # Not included in the original document.
    "Heavy Freezing Rain" : 123, # Not included in the original document.
    "Light Rain with Thunder" : 124, # Not included in the original document.
    "Light Snow" : 160, # Not included in the original document.
    "Heavy Snow" : 161, # Not included in the original document.
    "Mist" : 200, # Not included in the original document.
    "Thunder and Hail" : 400 # Not included in the original document.
}

In [12]:
# Function to get weather codes and handle combined conditions.
def get_weather_code(condition):
    # Split the combined conditions on the forward slash (/).
    conditions = condition.split(" / ")
    
    # Look up codes for each part of the combined condition.
    codes = [weather_codes.get(cond.strip(), None) for cond in conditions]
    
    # If any condition is not found, return None.
    if None in codes:
        return np.nan
    
    # Concatenate codes if there are multiple.
    if len(codes) > 1:
        # Concatenate the codes as strings and convert to float.
        concatenated_code = "".join(map(str, codes))
        return float(concatenated_code)
    
    # Return the single code as a float.
    return float(codes[0])

# Apply the function to the DataFrame.
df["weather_code"] = df["weather_condition"].apply(get_weather_code)

## Translating *"wind"* to the Relevant Custom Wind Code

In [13]:
# Investigate what unique values there are in the column, finding their length, and ordering them in a list.
print("List of unique values in column:", df["wind"].unique())

List of unique values in column: ['S' 'SW' 'WSW' 'SSW' 'CALM' 'NW' 'VAR' 'SSE' 'NNW' 'WNW' 'W' 'ENE' 'E'
 'N' 'NE' 'NNE' 'SE' 'ESE' nan]


In [14]:
# Wind codes are assigned in clockwise order after "CALM" and "VAR".
wind_codes = {
    "CALM" : 0,
    "VAR" : 1, # Variable.
    "N" : 2,
    "NNE" : 3,
    "NE" : 4,
    "ENE" : 5,
    "E" : 6,
    "ESE" : 7,
    "SE" : 8,
    "SSE" : 9,
    "S" : 10,
    "SSW" : 11,
    "SW" : 12,
    "WSW" : 13,
    "W" : 14,
    "WNW" : 15,
    "NW" : 16,
    "NNW" : 17
}

In [15]:
df["wind_code"] = df["wind"].map(wind_codes)

In [16]:
# Saving the dataframe to a new .csv file.
file_path = "historical_weather_data_with_codes.csv"

# Save the dataframe as a .csv file.
df.to_csv(file_path, index = False)